In [43]:
import mysql.connector 

mydb = mysql.connector.connect(
    host="localhost",
    port=3307,
    user="app_user",
    password="app_pass",
    database="store"
)

print(mydb)

In [42]:
cursor = mydb.cursor()
cursor.execute("SELECT * FROM products;")
result = cursor.fetchone()

print("Connected to:", result)

cursor.close()
mydb.close()

Connected to: None


In [44]:
cursor = mydb.cursor()
cursor.execute("SELECT * FROM transactions;")
result = cursor.fetchone()

print("Connected to:", result)

cursor.close()
mydb.close()

Connected to: (101, 5, 2001, datetime.datetime(2025, 10, 27, 10, 30), 'card', Decimal('20.00'), Decimal('0.00'))
